In [95]:
import pandas as pd
import csv
import numpy as np
pd.set_option('display.max_rows', 1000)
from sklearn.metrics import classification_report

gold = pd.read_csv("./../data/conll/test_only_space-spans.conll",
                   sep="\t", quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str').dropna()
gold['End'] = gold['End'].apply(lambda x: int(x[:-2]))

pred = pd.read_csv("..\\..\\..\\..\\..\\Downloads\\Flair\\transf_multilingual_crf_context_rnn_true\\test.tsv",
                   sep=" ", names=['token', 'gold','pred'], quoting = csv.QUOTE_NONE, encoding="utf-8", dtype='str')

In [106]:
df = pd.concat([gold.reset_index(), pred], axis=1)
df.loc[df.Doc_Id == "1096118458884988930"]

,index,Doc_Id,Sent_Id,Token_Id,Word,Tag,Begin,End,token,gold,pred
417,432,1096118458884988930,16,1,Último,O,0,6,Último,O,O
418,433,1096118458884988930,16,2,día,O,7,10,día,O,O
419,434,1096118458884988930,16,3,entrenamiento,O,11,24,entrenamiento,O,O
420,435,1096118458884988930,16,4,#cross,O,25,31,#cross,O,O
421,436,1096118458884988930,16,5,#VegaDeCantimpalos,O,32,50,#VegaDeCantimpalos,O,O
422,437,1096118458884988930,16,6,este,O,51,55,este,O,O
423,438,1096118458884988930,16,7,domingo.,O,56,64,domingo.,O,O
424,439,1096118458884988930,16,8,#puestadesol,O,65,77,#puestadesol,O,O
425,440,1096118458884988930,16,9,por,O,78,81,por,O,O
426,441,1096118458884988930,16,10,la,O,82,84,la,O,O


In [97]:
print(len(gold), len(pred))

75700 75700


In [98]:
print(classification_report(df.gold, df.pred, labels=["B-MISC","I-MISC"]))

              precision    recall  f1-score   support

      B-MISC       0.94      0.96      0.95      4259
      I-MISC       0.90      0.85      0.87      1570

   micro avg       0.93      0.93      0.93      5829
   macro avg       0.92      0.90      0.91      5829
weighted avg       0.93      0.93      0.93      5829



In [105]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import csv
import numpy as np
import argparse
from tqdm import tqdm

from sklearn.metrics import classification_report


def peek_stack(stack):
    if stack:
        return stack[-1]    # this will get the last element of stack
    else:
        return None

def merge_spans2(spans, words, debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    print(spans.shape)
    if spans.shape[0] == 1:
        return spans, words
    stack = []
    merged_words = []
    for s, w in zip(spans.reshape(1,-1)[0], words):
        if debug : print(s)
            
        if len(stack) == 0:
            stack.append(s)
            merged_words.append(w)
        else:
            if s - peek_stack(stack) == 1:
                stack.pop()
            else:
                stack.append(s)
        if debug : print("current stack: ", stack)
    stack = np.asarray(stack)
    return stack.reshape(-1, 2), merged_words
    
#merge_spans2(debug=True)    

# def create_submission_file(args):

path = "./../data/socialdisner_v3/train-valid-txt-files/validation/"

validation = gold

predictions = pred

# validation['pred'] = predictions.pred

print(classification_report(predictions.gold, predictions.pred, labels=['B-MISC', 'I-MISC']))

print('converting to submission format')
with open("./../submissions/testgold.tsv", "w", encoding="utf-8") as fp:
    print('tweets_id','begin','end','type','extraction',sep='\t', file=fp)
    for i in tqdm(range(1, len(validation.Sent_Id.drop_duplicates()) + 1)):
        info = df[df.Sent_Id.astype(int) == i]
        #print(info)
        l = info[info.gold != 'O']
        print(l)
        if len(l) == 0: continue
        docid = l.Doc_Id.values[0]
        print(docid)
        f = open(f'{path}{docid}.txt', 'r', encoding="utf-8")
        # replace tab with space then strip extra spaces
        text = f.read().replace('\n',' ').strip()
        annoying_chars = ["️","⁩","⁦","2⃣"," ", " ", " ", "⁩", "⁦", "‍", " "]
        for char in annoying_chars:
            text = text.replace(char, '_').strip()
        #print(text)
        #print('*'*20)
        #print(l.Doc_Id.values[0], l.begin.values[0], l.end.values[0], l.pred.values[0])
        spans = [[s,e] for s,e in zip(l.Begin.astype(int), l.End.astype(int))]
        words = [w for w in l.Word]
        print(spans, words)
        spans,mwords = merge_spans3(spans, words)
        #print(spans)
        for span,mw in zip(spans,mwords):
#             aman = text[span[0]: span[1]]
#             for char in annoying_chars:
#                 aman = aman.replace(char, '_').strip()
            print(docid, span[0], span[1], 'ENFERMEDAD', mw, sep='\t', file=fp)

#         "print("#"*20)
        #break


# create_submission_file(args)

              precision    recall  f1-score   support

      B-MISC       0.94      0.96      0.95      4259
      I-MISC       0.90      0.85      0.87      1570

   micro avg       0.93      0.93      0.93      5829
   macro avg       0.92      0.90      0.91      5829
weighted avg       0.93      0.93      0.93      5829

converting to submission format


  0%|▎                                                                                | 9/2500 [00:00<00:30, 82.72it/s]

   index               Doc_Id Sent_Id Token_Id   Word     Tag Begin  End  \
7      7  1091374539601309697       1        8  gripe  B-MISC    38   43   

   token    gold    pred  
7  gripe  B-MISC  B-MISC  
1091374539601309697
[[38, 43]] ['gripe']
[38 43] gripe
1 1
    index               Doc_Id Sent_Id Token_Id          Word     Tag Begin  \
28     29  1091997525605826560       2       13  discapacidad  B-MISC    77   

    End         token    gold    pred  
28   89  discapacidad  B-MISC  B-MISC  
1091997525605826560
[[77, 89]] ['discapacidad']
[77 89] discapacidad
1 1
    index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  End  \
77     79  1092363545017286656       3       27  ansiedad  B-MISC   153  161   

       token    gold    pred  
77  ansiedad  B-MISC  B-MISC  
1092363545017286656
[[153, 161]] ['ansiedad']
[153 161] ansiedad
1 1
     index               Doc_Id Sent_Id Token_Id              Word     Tag  \
100    103  1092416725902073856       4        5    

[232 257] #hemianopsia #hemiparesia
2 2
     index               Doc_Id Sent_Id Token_Id         Word     Tag Begin  \
478    494  1097178488421916672      17       19       cáncer  B-MISC   106   
479    495  1097178488421916672      17       20     infantil  I-MISC   113   
483    499  1097178488421916672      17       24  @criscancer  B-MISC   134   

     End        token    gold    pred  
478  112       cáncer  B-MISC  B-MISC  
479  121     infantil  I-MISC  I-MISC  
483  145  @criscancer  B-MISC  B-MISC  
1097178488421916672


  1%|▉                                                                               | 28/2500 [00:00<00:30, 80.27it/s]

...

       index               Doc_Id Sent_Id Token_Id       Word     Tag Begin  \
74104  76551  1495512989998456832    2447       31     ataque  B-MISC   145   
74105  76552  1495512989998456832    2447       32         de  I-MISC   152   
74106  76553  1495512989998456832    2447       33  ansiedad.  I-MISC   155   

       End      token    gold    pred  
74104  151     ataque  B-MISC  B-MISC  
74105  154         de  I-MISC  I-MISC  
74106  164  ansiedad.  I-MISC  I-MISC  
1495512989998456832
[[145, 151], [152, 154], [155, 164]] ['ataque', 'de', 'ansiedad.']
[145 164] ataque de ansiedad.
1 1
       index               Doc_Id Sent_Id Token_Id       Word     Tag Begin  \
74115  76563  1495580080520765441    2448        7  hepatitis  B-MISC    38   
74116  76564  1495580080520765441    2448        8         A.  I-MISC    48   

       End      token    gold    pred  
74115   47  hepatitis  B-MISC  B-MISC  
74116   50         A.  I-MISC  I-MISC  
1495580080520765441
[[38, 47], [48, 50]] ['h

 99%|█████████████████████████████████████████████████████████████████████████████▎| 2478/2500 [00:26<00:00, 91.74it/s]

[[6, 14]] ['ansiedad']
[ 6 14] ansiedad
1 1
       index               Doc_Id Sent_Id Token_Id      Word     Tag Begin  \
74695  77162  1496532424712462341    2467        4    ataque  B-MISC    12   
74696  77163  1496532424712462341    2467        5        de  I-MISC    19   
74697  77164  1496532424712462341    2467        6  ansiedad  I-MISC    22   

       End     token    gold    pred  
74695   18    ataque  B-MISC  B-MISC  
74696   21        de  I-MISC  I-MISC  
74697   30  ansiedad  I-MISC  I-MISC  
1496532424712462341
[[12, 18], [19, 21], [22, 30]] ['ataque', 'de', 'ansiedad']
[12 30] ataque de ansiedad
1 1
       index               Doc_Id Sent_Id Token_Id           Word     Tag  \
74762  77230  1496539158780821516    2468       20  conjuntivitis  B-MISC   

      Begin  End          token    gold    pred  
74762    97  110  conjuntivitis  B-MISC  B-MISC  
1496539158780821516
[[97, 110]] ['conjuntivitis']
[ 97 110] conjuntivitis
1 1
       index               Doc_Id Sent_Id T

100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:26<00:00, 93.27it/s]

[[15, 20]] ['covid']
[15 20] covid
1 1
       index               Doc_Id Sent_Id Token_Id                     Word  \
75194  77680  1497846585442672644    2486        9                 insomnio   
75202  77688  1497846585442672644    2486       17                 ansiedad   
75214  77700  1497846585442672644    2486       29           #endometriosis   
75215  77701  1497846585442672644    2486       30    #endometriosiscronica   
75216  77702  1497846585442672644    2486       31  #endometriosisargentina   

          Tag Begin  End                    token    gold    pred  
75194  B-MISC    40   48                 insomnio  B-MISC       O  
75202  B-MISC    78   86                 ansiedad  B-MISC  B-MISC  
75214  B-MISC   186  200           #endometriosis  B-MISC  B-MISC  
75215  B-MISC   201  222    #endometriosiscronica  B-MISC  B-MISC  
75216  B-MISC   223  246  #endometriosisargentina  B-MISC  B-MISC  
1497846585442672644
[[40, 48], [78, 86], [186, 200], [201, 222], [223, 246]] [

In [38]:
merge_spans2(spans= [[39, 46], [47, 52], [55, 62]], words = ['autismo', 'TEA', 'autismo'], debug=True)

[[39, 46], [47, 52], [55, 62]]
(3, 2)
39
current stack:  [39]
46
current stack:  [39, 46]
47
current stack:  [39]


ValueError: cannot reshape array of size 1 into shape (2)

In [100]:
def merge_spans3(spans=[[39, 46], [47, 52], [55, 62]], words = ['bcbc', 'aa', 'autismo'], debug=False):
    
    if debug : print(spans)
    spans = np.asarray(spans)
    stack = []
    s2w = {}
    
    for s in spans.reshape(1,-1)[0]:
        
        if debug : print('==>', s)
        if len(stack) == 0:
            stack.append(s)
        else:
            if s - peek_stack(stack) == 1:
                stack.pop()
            else:
                stack.append(s)
        if debug : print("current stack: ", stack)
    stack = np.asarray(stack).reshape(-1,2)
    
    for s,word in zip(spans,words):
        for ss in s:
            s2w[ss] = word
    word_list = []
    for s in stack:
        sw = []
        #print(s[0], s[1])
        idx = [n for n in s2w.keys() if (n>=s[0] and n<=s[1])]
        widx = [s2w[i] for i in idx]
        fin_word = []
        for w in widx :
            if w not in fin_word:
                fin_word.append(w)
        #print(widx)
        print(s, ' '.join(fin_word))
        word_list.append(' '.join(fin_word))
    
    print(len(stack), len(word_list))
        
    return stack, word_list
    
merge_spans3(debug=True)

[[39, 46], [47, 52], [55, 62]]
==> 39
current stack:  [39]
==> 46
current stack:  [39, 46]
==> 47
current stack:  [39]
==> 52
current stack:  [39, 52]
==> 55
current stack:  [39, 52, 55]
==> 62
current stack:  [39, 52, 55, 62]
[39 52] bcbc aa
[55 62] autismo
2 2


(array([[39, 52],
        [55, 62]]),
 ['bcbc aa', 'autismo'])

Traceback (most recent call last):
  File "../scripts/eval_script.py", line 78, in <module>
    run_eval(args)
  File "../scripts/eval_script.py", line 9, in run_eval
    mention = pd.read_csv('./data/socialdisner_v3/mentions.tsv', sep='\t')
  File "C:\Users\34645\miniconda3\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\34645\miniconda3\lib\site-packages\pandas\io\parsers\readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\34645\miniconda3\lib\site-packages\pandas\io\parsers\readers.py", line 482, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Users\34645\miniconda3\lib\site-packages\pandas\io\parsers\readers.py", line 811, in __init__
    self._engine = self._make_engine(self.engine)
  File "C:\Users\34645\miniconda3\lib\site-packages\pandas\io\parsers\readers.py", line 1040, in _make_engine
    return mapping[engine](self.f, **self.options)  